# Face Recognition using Transfer Learning

## 1.Data Collection

In [ ]:
import cv2
import numpy as np

# Load HAARCASCADE face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Load functions
def face_extractor(img):
    # Function detects faces and returns the cropped face
    # If no face detected, it returns the input image
    
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if faces is ():
        return None
    
    # Crop all faces found
    for (x,y,w,h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

# Initialize Webcam
cap = cv2.VideoCapture(0)
count = 0

# Collect required samples of your face from webcam input (train=960,valid=240)
while True:

    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        # Save file in specified directory with unique name
        #change directoy name for person and separate folder for train and valid
        file_name_path = '/folder/to/save/images' + str(count) + '.jpg' 
        cv2.imwrite(file_name_path, face)

        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Face Cropper', face)
        
    else:
        print("Face not found")
        pass

    if cv2.waitKey(1) == 13 or count == 960: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()      
print("Collecting Samples Complete")

## 2. Importing VGG16 Model and freezing Layers

In [ ]:
from keras.applications import VGG16
#Resizing images to 64 * 64 pixels

img_rows = 64
img_cols = 64
vgg16 = VGG16(weights = 'imagenet',
             include_top = False,
             input_shape = (img_rows,img_cols, 3))

#Freezing layers
for layer in vgg16.layers:
    layer.trainable = False
# Printing Layers
for (i, layer) in enumerate(vgg16.layers):
    print(str(i) + " "+ layer.__class__.__name__, layer.trainable)

## 3. Adding a Top model (Fully connected Deep Learning Network)

In [ ]:
def addTopModel(bottom_model , num_class, du=256):
    """ creating top models that will be placed on top of the bottom model """
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(du,activation = "relu")(top_model)
    top_model = Dense(512,activation = "relu")(top_model)
    top_model = Dense(1024,activation = "relu")(top_model)
    top_model = Dropout(0.3)(top_model)
    top_model = Dense(num_class, activation = "softmax")(top_model)
    return top_model

## 4. Image Augmentation to increase dataset size

In [ ]:
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator

train_data_dir = "/root/facedata/train"
validation_data_dir = "/root/facedata/valid"

train_datagen = ImageDataGenerator(rescale = 1./255,
                                  rotation_range = 30,
                                  width_shift_range = 0.2,
                                   height_shift_range= 0.2,
                                   horizontal_flip= True,
                                   fill_mode='nearest')
valid_datagen = ImageDataGenerator(rescale = 1./255)

# Specify batch size according to system RAM
train_batchsize = 10 
valid_batchsize = 10

train_gen = train_datagen.flow_from_directory(train_data_dir,
                                             target_size=(img_rows,img_cols),
                                             batch_size=train_batchsize,
                                             class_mode= 'categorical')

valid_gen = train_datagen.flow_from_directory(validation_data_dir,
                                             target_size=(img_rows,img_cols),
                                             batch_size=valid_batchsize,
                                             class_mode= 'categorical',
                                             shuffle=False)
num_class = 5

model_head= addTopModel(vgg16,num_class)
model = Model(inputs=vgg16.input, outputs=model_head)

In [ ]:
model.summary()

## 5. Training model

In [ ]:
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    "Face_Recognition.h5",
    monitor="val_loss",
    mode='min',
    verbose=1
)
earlystop = EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=5,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', 
                              factor=0.2,
                              patience=5, 
                              min_lr=0.00001)
callbacks = [earlystop,checkpoint,reduce_lr]
model.compile(loss='categorical_crossentropy', 
             optimizer = RMSprop(lr = 0.0001),
             metrics = ['accuracy'])
train_sample = 960
valid_sample = 240
epochs = 25
batch_size = 10

history =model.fit_generator(train_gen,
                            steps_per_epoch = train_sample // batch_size,
                            epochs = epochs,
                            callbacks = callbacks,
                            validation_data = valid_gen,
                            validation_steps = valid_sample // batch_size)
model.save("Face_Recognition.h5")

## 6. Testing Model in Real Time using webcam

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import matplotlib.pyplot as plt
classifier = load_model("Face_Recognition.h5")

In [ ]:
faceCascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

video_capture = cv2.VideoCapture(0)


while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.FONT_HERSHEY_SIMPLEX
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        img=frame[y:y+h, x:x+w]
    #preprocess
        #img = image.load_img(img,target_size=(64,64,3))
        img = cv2.resize(img,(64,64))
        img = image.img_to_array(img)
        img = np.expand_dims(img,axis=0)
        result = np.argmax(classifier.predict(img,1,verbose=0),axis=1)

    # Display the resulting frame {0:'Person 1', 1:'Person 2', 2:'Person 3', 3:'Person 3', 4:'Person 5'}
    if result == 0:
        person = "Person 1"
    elif result == 1:
        person = "Person 2"
    elif result == 2 :
        person == "Person 3"
    elif result== 3:
        person = "Person 4"
    elif result == 4:
        person = "Person 5"
    cv2.putText(frame,str(person) , (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()